In [2]:
import os

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, DataParallel
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset, DataListLoader
from ModelFunctions import train, evaluate, inference

In [3]:
import sys
sys.path.append(os.path.expanduser('../ocpmodels/models'))
sys.path.append(os.path.expanduser('../../ocp_airi'))

from spinconv_with_embeds import spinconv

In [4]:
#вызывается каждый раз, когда датасет отдаёт элемент (систему)
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер; надо писать свою функцию для каждой сети
def preprocessing(system):
    keys = ['pos', 'atomic_numbers', 'cell', 'natoms']
    features_dict = {}
    for key in keys:
        features_dict[key] = system[key]
    return Data(**features_dict)

In [5]:
#config
batch_size = 35
num_workers = 0

features_cols = ['feature_1']

target_col = 'y_relaxed'
lr = 0.001
epochs = 20

In [6]:
# #чтобы тензор по умолчанию заводился на куде
# if torch.cuda.is_available():
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')
#     print('cuda')

In [7]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cuda


In [8]:
#инициализируем тренировочный датасети и тренировочный итератор
train_dataset_file_path= os.path.expanduser("../../ocp_datasets/data/is2re/all/train/data.lmdb")

training_set = Dataset(train_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
training_generator = DataListLoader(training_set, batch_size=batch_size)

In [9]:
#инициализируем валидационный датасет и валидационный итератор
val_dataset_file_path = os.path.expanduser("../../ocp_datasets/data/is2re/all/val_ood_both/data.lmdb")

valid_set = Dataset(val_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
valid_generator = DataListLoader(valid_set, batch_size=batch_size, num_workers=num_workers)

In [10]:
try:
    lmdb_dataset(train_dataset_file_path).describe()
except:
    pass

total entries: 460328
info for item: 0
edge_index:...............<class 'torch.Tensor'>..... [2, 2964]
pos:......................<class 'torch.Tensor'>.....   [86, 3]
cell:.....................<class 'torch.Tensor'>..... [1, 3, 3]
atomic_numbers:...........<class 'torch.Tensor'>.....      [86]
natoms:...................       <class 'int'>.....        86
cell_offsets:.............<class 'torch.Tensor'>..... [2964, 3]
force:....................<class 'torch.Tensor'>.....   [86, 3]
distances:................<class 'torch.Tensor'>.....    [2964]
fixed:....................<class 'torch.Tensor'>.....      [86]
sid:......................       <class 'int'>.....   2472718
tags:.....................<class 'torch.Tensor'>.....      [86]
y_init:...................     <class 'float'>.....    6.2825
y_relaxed:................     <class 'float'>.....   -0.0256
pos_relaxed:..............<class 'torch.Tensor'>.....   [86, 3]


In [11]:
#model
model = spinconv(None, None, 1, otf_graph=True, regress_forces=False)
model = DataParallel(model)

#optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.L1Loss()

#переносим на куду если она есть
model = model.to(device)
criterion = criterion.to(device)

In [12]:
timestamp = str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

print(timestamp)

2021-09-28-15-44-11


In [13]:
#tensorboard writer, при первом запуске надо руками сделать папку для логов

# server
#log_folder_path = "../../ocp_results/logs/tensorboard/out_base_model"

# colab
# log_folder_path = "/content/drive/MyDrive/ocp_results/logs/tensorboard/out_base_model"

# user_specific 
log_file_path = "../logs/tensorboard_airi"

writer = SummaryWriter(log_file_path + '/' + timestamp)

In [14]:
%%time
logfile_str = {
    "train_dataset_file_path": train_dataset_file_path,
    "val_dataset_file_path": val_dataset_file_path,
    "features_cols": features_cols,
    "target_col": target_col,
    "batch_size": batch_size,
    "num_workers": num_workers,
    "epochs": epochs,
    "lr": lr
}

#граф модели
try:
    #trace_system = dict(list(next(iter(training_generator))[0]))
    writer.add_graph(model, trace_system)
except:
    print('no graph')
writer.add_text(timestamp, str(logfile_str))

no graph
CPU times: user 362 µs, sys: 98 µs, total: 460 µs
Wall time: 370 µs


## Training

In [ ]:
%%time
loss = []
loss_eval = []

print(timestamp)
print(f'Start training model {str(model)}')
for i in range(epochs):
    loss.append(train(model, training_generator, optimizer, criterion, epoch=i, writer=writer, device=device))
    loss_eval.append(evaluate(model, valid_generator, criterion, epoch=i, writer=writer, device=device))

In [ ]:
evaluate(model, training_generator, criterion, epoch=-1, writer=writer, device=device)

epoch -1 evaluation


In [ ]:
m(next(iter(training_generator)))

tensor([[ -6.1095],
        [ -6.0353],
        [ -4.7002],
        [ -4.4152],
        [ -4.7612],
        [-13.9889],
        [-14.8332],
        [ -4.9109],
        [ -4.6978],
        [ -4.9114],
        [ -8.0928],
        [ -5.2591],
        [ -8.7300],
        [ -4.0574],
        [ -3.9197],
        [ -6.9675],
        [-12.7758],
        [ -7.1721],
        [ -7.4004],
        [ -5.4814],
        [ -5.4049],
        [ -5.2608],
        [-13.7024],
        [ -5.2661],
        [ -6.1901],
        [ -4.9087],
        [ -2.3582],
        [ -4.9832],
        [ -2.4268],
        [ -4.8415],
        [ -6.8876],
        [ -3.2091],
        [ -4.8425],
        [ -6.0368],
        [ -5.2641]], device='cuda:0', grad_fn=<ScatterAddBackward>)